# Data Enrichment Project

# Imports & Data Cleaning


In [43]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, time, json, math
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
# defining function for movie rating
# CITING: Using TMDB API from LP
"""Adapated from https://github.com/celiao/tmdbsimple"""

def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    
    info = movie.info()
    
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
            
    return info

In [3]:
# defining function for writing json
# CITING: Efficient TMDB API calls from LP
"""Adapted from https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

def write_json(new_data, filename):
    
    with open(filename,'r+') as file:
        file_data = json.load(file)
        
        if(type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
            
        file.seek(0)
        json.dump(file_data,file)

In [4]:
basic_gz = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_gz = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
akas_gz = 'https://datasets.imdbws.com/title.akas.tsv.gz'

In [5]:
basics = pd.read_csv(basic_gz, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_gz, sep='\t', low_memory=False)
akas = pd.read_csv(akas_gz, sep='\t', low_memory=False)

**Basics DF Data Cleanse**

In [6]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [7]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9317803 entries, 0 to 9317802
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 639.8+ MB


In [8]:
# replacing null values with np.nan
basics = basics.replace({'\\N':np.nan})
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [9]:
# eliminate movies that are null for runtimeMinutes
basics = basics.dropna(subset=['runtimeMinutes'])
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2525791 entries, 0 to 9317802
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 192.7+ MB


In [10]:
# eliminate movies that are null for genres
basics = basics.dropna(subset=['genres'])
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2457786 entries, 0 to 9317802
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 187.5+ MB


In [11]:
# keep only titleType==Movie
basics['titleType'].value_counts()

tvEpisode       1111586
short            580384
movie            370476
video            176183
tvMovie           88793
tvSeries          87445
tvSpecial         16853
tvMiniSeries      16321
tvShort            9437
videoGame           308
Name: titleType, dtype: int64

In [12]:
basics = basics[basics.titleType == 'movie']
basics['titleType'].value_counts()

movie    370476
Name: titleType, dtype: int64

In [13]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1172,tt0001184,movie,Don Juan de Serrallonga,Don Juan de Serrallonga,0,1910,NaN,58,"Adventure,Drama"


In [14]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370476 entries, 8 to 9317753
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          370476 non-null  object
 1   titleType       370476 non-null  object
 2   primaryTitle    370476 non-null  object
 3   originalTitle   370476 non-null  object
 4   isAdult         370476 non-null  object
 5   startYear       364551 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  370476 non-null  object
 8   genres          370476 non-null  object
dtypes: object(9)
memory usage: 28.3+ MB


In [15]:
# keep startYear 2000-2022
basics = basics.dropna(subset=['startYear'])
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364551 entries, 8 to 9317753
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          364551 non-null  object
 1   titleType       364551 non-null  object
 2   primaryTitle    364551 non-null  object
 3   originalTitle   364551 non-null  object
 4   isAdult         364551 non-null  object
 5   startYear       364551 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  364551 non-null  object
 8   genres          364551 non-null  object
dtypes: object(9)
memory usage: 27.8+ MB


In [16]:
basics['startYear'] = basics['startYear'].astype(int)
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364551 entries, 8 to 9317753
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          364551 non-null  object
 1   titleType       364551 non-null  object
 2   primaryTitle    364551 non-null  object
 3   originalTitle   364551 non-null  object
 4   isAdult         364551 non-null  object
 5   startYear       364551 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  364551 non-null  object
 8   genres          364551 non-null  object
dtypes: int32(1), object(8)
memory usage: 26.4+ MB


In [17]:
basics = basics.drop(basics[basics['startYear']<2000].index)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61095,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
66310,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006,NaN,47,Documentary
67641,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


In [18]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218341 entries, 13079 to 9317753
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          218341 non-null  object
 1   titleType       218341 non-null  object
 2   primaryTitle    218341 non-null  object
 3   originalTitle   218341 non-null  object
 4   isAdult         218341 non-null  object
 5   startYear       218341 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  218341 non-null  object
 8   genres          218341 non-null  object
dtypes: int32(1), object(8)
memory usage: 15.8+ MB


In [19]:
# Eliminate movies that include "Documentary" in genre
documentary_pos = basics['genres'].str.contains('documentary',case=False)
basics = basics[~documentary_pos]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61095,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67641,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77935,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [20]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144581 entries, 34793 to 9317652
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          144581 non-null  object
 1   titleType       144581 non-null  object
 2   primaryTitle    144581 non-null  object
 3   originalTitle   144581 non-null  object
 4   isAdult         144581 non-null  object
 5   startYear       144581 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  144581 non-null  object
 8   genres          144581 non-null  object
dtypes: int32(1), object(8)
memory usage: 10.5+ MB


In [21]:
# Keep only US movies
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34793      True
61095      True
67641      True
77935      True
86771      True
           ... 
9317475    True
9317484    True
9317523    True
9317568    True
9317652    True
Name: tconst, Length: 144581, dtype: bool

In [22]:
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61095,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67641,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77935,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [23]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143929 entries, 34793 to 9317652
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          143929 non-null  object
 1   titleType       143929 non-null  object
 2   primaryTitle    143929 non-null  object
 3   originalTitle   143929 non-null  object
 4   isAdult         143929 non-null  object
 5   startYear       143929 non-null  int32 
 6   endYear         0 non-null       object
 7   runtimeMinutes  143929 non-null  object
 8   genres          143929 non-null  object
dtypes: int32(1), object(8)
memory usage: 10.4+ MB


**AKA's DF Data Cleanse**

In [24]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [25]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33596624 entries, 0 to 33596623
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.0+ GB


In [26]:
# Replace "\N" with np.nan
akas = akas.replace({'\\N':np.nan})
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [27]:
# keep only US movies
akas['region'].value_counts()

JP    4010533
FR    4009403
DE    4006050
IN    3941036
ES    3932455
       ...   
CC          1
TV          1
NU          1
PW          1
NR          1
Name: region, Length: 247, dtype: int64

In [28]:
akas = akas.dropna(subset=['region'])
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31729920 entries, 0 to 33596622
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


In [29]:
is_us = akas['region'].str.contains('US',case=False)
akas = akas[is_us]
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [30]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1361884 entries, 5 to 33596368
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1361884 non-null  object
 1   ordering         1361884 non-null  int64 
 2   title            1361884 non-null  object
 3   region           1361884 non-null  object
 4   language         3698 non-null     object
 5   types            965093 non-null   object
 6   attributes       45220 non-null    object
 7   isOriginalTitle  1360509 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.5+ MB


**Ratings DF Data Clease**

In [31]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1920
1,tt0000002,5.8,260
2,tt0000003,6.5,1727
3,tt0000004,5.6,173
4,tt0000005,6.2,2542


In [32]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239729 entries, 0 to 1239728
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1239729 non-null  object 
 1   averageRating  1239729 non-null  float64
 2   numVotes       1239729 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.4+ MB


In [33]:
# Keep only US movies
keepers1 = ratings['tconst'].isin(akas['titleId'])
keepers1

0           True
1           True
2          False
3          False
4           True
           ...  
1239724    False
1239725     True
1239726    False
1239727    False
1239728    False
Name: tconst, Length: 1239729, dtype: bool

In [34]:
ratings = ratings[keepers1]
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1920
1,tt0000002,5.8,260
4,tt0000005,6.2,2542
5,tt0000006,5.1,175
6,tt0000007,5.4,797


In [35]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472281 entries, 0 to 1239725
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         472281 non-null  object 
 1   averageRating  472281 non-null  float64
 2   numVotes       472281 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.4+ MB


**Saving csv file**

In [39]:
basics.to_csv('Data/title.basics.csv.gz',compression='gzip',index=False)

In [40]:
basics = pd.read_csv('Data/title.basics.csv.gz',low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


**Loading API Credentials**

In [36]:
with open('/Users/chris/.secret/tmdb_api.json','r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [37]:
tmdb.API_KEY = login['api-key']

**Designating folder for API data**

In [38]:
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title.akas.tsv.gz', 'title.basics.tsv.gz', 'title.ratings.tsv.gz']

**Loading in dataframe**

In [41]:
basics = pd.read_csv('Data/title.basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [42]:
# defining lists of years to extract & errors list
YEARS_TO_GET = [2000,2001]
errors = []

In [45]:
# starting outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='Years', position=0):
    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

Years:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
# saving new year as current df
df = basics.loc[ basics['startYear']==YEAR].copy()
movie_ids = df['tconst'].copy()

In [48]:
previous_df = pd.read_json(JSON_FILE)

In [50]:
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [51]:
# starting inner loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                              desc=f'Movies from {YEAR}',
                              position=1,
                              leave=True):
    try:
        temp = get_movie_with_rating(movie_id)
        write_json(temp,JSON_FILE)
        time.sleep(0.02)
        
    except Exception as e:
        errors.append([movie_id,e])

Movies from 2001:   0%|          | 0/2844 [00:00<?, ?it/s]